# This code is written for the extraction of last column of                               
# the each patients for the base visit.
# Written by Zainab Nazari, last modify: 18.05.2023 15:46

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import functools

 We put the main path to the folder where 'counts' files are there.


In [2]:
path1=Path("/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir2/counts/")

 We read the file: in the bl.txt is written the name of all files.

In [3]:
bl_files=pd.read_csv(path1/"n_bl.txt", header=None )

In [4]:
len(bl_files)

5

 We define a function where we can take the second phrase seperated by dot. The second phrase 
is the patient ID. So with this functin we want to get the patient IDs from their file's name

In [5]:
def function_names(fname):
    tokens=fname.split('.')
    return tokens[1] 

In [6]:
def merge(a,b):
    a.iloc[:,-1:].reset_index(drop=False).merge(b.iloc[:,-1:].reset_index(drop=True), left_index=True, right_index=True)
    return a

In [7]:
def merge_df(a,b):
    return a.reset_index(drop=True).merge(b.iloc[:,-1:].reset_index(drop=True),left_index=True, right_index=True)

 here we go through all file names and extract their patient IDs and put into a list
 So here we have the list of patient IDs for those whom they have put the data for their base visit

In [8]:
bl_list=[]
for i in range(len(bl_files)):
    bl_list=bl_list+[function_names(bl_files.iloc[i][0])]

In [9]:
bl_list

['3016', '3020', '3023', '3024', '3026']

 here we read all the files with with base visit from the count folder where we have all the files
 for all the patients and all the visit.
 here we have a list of all files with base visit.

In [10]:
%%time
list_bl_files=[]
for i in range(len(bl_files)):
    list_bl_files=list_bl_files+[pd.read_csv(path1/bl_files.iloc[i][0],skiprows=1,delimiter='\t', index_col=[0])]

CPU times: user 2.16 s, sys: 278 ms, total: 2.43 s
Wall time: 2.43 s


In [11]:
# here  we take the last column of each file? 
for i in range(len(bl_list)):
    list_bl_files[i].columns = list(list_bl_files[i].columns[:-1])+[bl_list[i]]

In [12]:
# first two columns we merege 
first_two_elements_merge_bl=list_bl_files[0].iloc[:,-1:].reset_index(drop=False).merge(list_bl_files[1].iloc[:,-1:].reset_index(drop=True), left_index=True, right_index=True)

In [13]:
list_bl_files_conv=[first_two_elements_merge_bl]+list_bl_files[2:]

del list_bl_files

In [14]:
merged_bl=functools.reduce(merge_df,list_bl_files_conv)
del list_bl_files_conv


In [15]:
merged_bl

,Geneid,3016,3020,3023,3024,3026
0,ENSG00000223972.4,20,7,5,3,1
1,ENSG00000227232.4,244,239,199,280,192
2,ENSG00000243485.2,1,0,0,0,2
3,ENSG00000237613.2,0,0,0,0,0
4,ENSG00000268020.2,2,0,0,0,0
...,...,...,...,...,...,...
57815,ENSG00000198695.2,600,219,53,98,113
57816,ENSG00000210194.1,69,31,10,11,9
57817,ENSG00000198727.2,560,426,88,626,987
57818,ENSG00000210195.2,45,40,10,64,45


In [16]:
merged_bl.to_csv("random_test_bl_matrix_counts_ir2.txt",  index=False)